# PROJECT:

In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res

In [2]:
df = pd.read_table('lpmc11.dat')
database = db.Database('lpmc', df)
df

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,1,0,0,1,4,3,1,1,1.0,1,...,0.109444,0.000000,0.055556,0.000000,0,0.059444,1.5,0.15,0.0,0.112150
1,13,1,1,1,4,3,1,5,0.0,1,...,0.241389,0.000000,0.122222,0.000000,0,0.132222,0.0,0.50,0.0,0.065126
2,19,4,0,1,3,3,6,5,0.0,1,...,0.222500,0.000000,0.312222,0.000000,0,0.221667,0.0,0.56,0.0,0.086466
3,20,5,1,0,4,3,1,5,0.0,1,...,0.381667,0.000000,0.062222,0.000000,0,0.117222,0.0,0.41,0.0,0.097156
4,39,9,2,0,4,3,1,5,0.0,1,...,0.146944,0.000000,0.225000,0.000000,0,0.200833,0.0,0.48,0.0,0.378976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,81035,17603,1,1,3,1,1,1,1.0,3,...,0.278056,0.216667,0.000000,0.000000,0,0.392222,2.4,0.98,0.0,0.547450
4996,81040,17604,2,0,3,1,1,2,0.0,3,...,0.264444,0.000000,0.353333,0.176667,1,0.288889,0.0,0.85,0.0,0.175000
4997,81045,17605,0,1,4,3,1,5,0.0,3,...,0.128889,0.000000,0.045833,0.000000,0,0.067778,0.0,0.17,0.0,0.024590
4998,81066,17608,0,3,3,3,6,1,1.0,3,...,0.092222,0.000000,0.389444,0.000000,0,0.193333,1.5,0.61,0.0,0.485632


# Model 0

**Variables:**

In [3]:
dur_walking = Variable('dur_walking') #time for walking
dur_cycling = Variable('dur_cycling') #time for cycling

dur_pt_access = Variable('dur_pt_access')
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int')

dur_pt = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int #total time for public transport
dur_driving = Variable('dur_driving')

cost_transit = Variable('cost_transit') #cost for public transport
cost_driving_fuel = Variable('cost_driving_fuel')
cost_driving_ccharge = Variable('cost_driving_ccharge')

cost_driving = cost_driving_fuel + cost_driving_ccharge #total cost for driving

travel_mode = Variable('travel_mode') #Choice

**Parameters:**

In [4]:
constant_walking = Beta('constant_walking', 0, None, None, 0)
constant_cycling = Beta('constant_cycling', 0, None, None, 0)
constant_pt = Beta('constant_pt', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

In [5]:
v_walk_Model0 = (
    constant_walking
    + beta_time * dur_walking
)

v_cycling_Model0 = (
    constant_cycling
    + beta_time * dur_cycling
)

v_PT_Model0 = (
    constant_pt
    + beta_cost * cost_transit
    + beta_time * dur_pt
)

v_driving_Model0 = (
    beta_cost * cost_driving
    + beta_time * dur_driving
)

In [6]:
V_Model0 = {1: v_walk_Model0, 2: v_cycling_Model0, 3: v_PT_Model0, 4: v_driving_Model0}
logprob_Model0 = models.loglogit(V_Model0, None, travel_mode)
biogeme_Model0 = bio.BIOGEME(database, logprob_Model0) #create the biogeme estimator
biogeme_Model0.modelName = 'logit_Model0'
res_Model0 = biogeme_Model0.estimate()

File biogeme.toml has been created


In [7]:
print(res_Model0.printGeneralStatistics())

Number of estimated parameters:	5
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4676.875
Likelihood ratio test for the init. model:	4509.193
Rho-square for the init. model:	0.325
Rho-square-bar for the init. model:	0.325
Akaike Information Criterion:	9363.751
Bayesian Information Criterion:	9396.337
Final gradient norm:	2.6771E-04
Nbr of threads:	8



In [8]:
res_Model0.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.156585,0.012667,-12.361233,0.0
beta_time,-5.226955,0.191905,-27.237207,0.0
constant_cycling,-2.485329,0.086252,-28.814839,0.0
constant_pt,0.726783,0.047157,15.411971,0.0
constant_walking,1.187012,0.078799,15.063738,0.0


The sign of the 2 generic coefficent leaves us satisfied. We know it's reasonable that utility is decreased when cost and/or travel time increase. It also seems that time impacts more the utility, since in asbolute value beta_time is higher than beta_cost. Concernig the ASCs, we normalized with respect to ASC_car so this is our baseline. We notice that ASC_walking is the highest so it seems that people, assuiming the same atrributes values, would choose this alternative. Anyway we see that, since the value of beta_time is high in absoulute value, and walking usually requires more time than the others, it will be penalized, as we expected. Cycling seems really disadvantaged, since its ASC is already low, and it has the same beta_time (due to generic coefficent assumption). Significance of value is confirmed by the t-test values that are all above the 1.96 threshold

# Model 1

We decide to split beta_time into 4 alternative specific parameters. Our choice is due to the fact that this coefficent is present in every utility function, and we think it can undergo a wider variation, since time perception is strictly related to the chosen travel mode.

In [9]:
constant_walking = Beta('constant_walking', 0, None, None, 0)
constant_cycling = Beta('constant_cycling', 0, None, None, 0)
constant_pt = Beta('constant_pt', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time_walking = Beta('beta_time_walking', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)

In [10]:
v_walk_Model1 = (
    constant_walking
    + beta_time_walking * dur_walking
)

v_cycling_Model1 = (
    constant_cycling
    + beta_time_cycling * dur_cycling
)

v_PT_Model1 = (
    constant_pt
    + beta_cost * cost_transit
    + beta_time_pt * dur_pt
)

v_driving_Model1 = (
    beta_cost * cost_driving
    + beta_time_driving * dur_driving
)

In [11]:
V_Model1 = {1: v_walk_Model1, 2: v_cycling_Model1, 3: v_PT_Model1, 4: v_driving_Model1}
logprob_Model1 = models.loglogit(V_Model1, None, travel_mode)
biogeme_Model1 = bio.BIOGEME(database, logprob_Model1)
biogeme_Model1.modelName = 'logit_Model1'
res_Model1 = biogeme_Model1.estimate()

In [12]:
print(res_Model1.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4372.911
Likelihood ratio test for the init. model:	5117.122
Rho-square for the init. model:	0.369
Rho-square-bar for the init. model:	0.368
Akaike Information Criterion:	8761.822
Bayesian Information Criterion:	8813.959
Final gradient norm:	2.8918E-03
Nbr of threads:	8



In [13]:
res_Model1.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.136269,0.013923,-9.787478,0.000000e+00
beta_time_cycling,-5.436177,0.492925,-11.028412,0.000000e+00
beta_time_driving,-5.794675,0.384636,-15.065358,0.000000e+00
beta_time_pt,-3.127377,0.243891,-12.822864,0.000000e+00
beta_time_walking,-8.161128,0.386290,-21.126960,0.000000e+00
constant_cycling,-2.560688,0.150700,-16.991966,0.000000e+00
constant_pt,-0.405922,0.069445,-5.845207,5.059401e-09
constant_walking,1.933359,0.129143,14.970653,0.000000e+00


As before, signs are reasonable. Concernign the values of the coefficents, we firstly notice that beta_cost reamined pretty much the same and it seems reasonable becuase people usually care how much money they spend rather than for what they do spend. The wide range of values we can see on beta_time strenghtens our initial assumption, in fact we see how beta_walking is really low, since walking requires phisycal effort, while the two alternatives that just require the individual's focus (driving and cycling) are pretty similar and this seems fine. In the end we see that beta_time_PT is instead the lowest in absolute value, and this is logic for us since when you are in public transport you are able to do other activities or just realx without having to mantain focus. Again, as before, ASC-walking is still bigger than car, as cycling is still the lowest. The only change we notice is the sign of ASC_pt, which is probably due to the value of the beta_time_pt we previously analyzed. Significance of value is confirmed by the t-test values that are all above the 1.96 threshold

In [14]:
res_Model1.likelihood_ratio_test(res_Model0, 0.01)

LRTuple(message='H0 can be rejected at level 1.0%', statistic=607.9289925585272, threshold=11.344866730144373)

To compare the 2 models we used the likelihood ratio test. This is doable since model 0 is a linear restriction of model 1 (assuming all the alternative specific beta_time equal). Our null-hypotesis is that the resricted model (in this case model 0) is the true model. We set the significance level of the test to 0,01 and as we can see from the result the null-hypotesis is rejected, hence our preferred model is model 1

# Model 2:

- Interaction of age with ASCs:

We select 'age' as socio-economic characteristic since we think it has a stornger correlation with the choice of a transport mode rather than sex, which is the other socio-ecomic characteristic we found in the dataset. In fact, if we assume two individuals have the same age, from our experience, their choice (putting all the other parametres equal) would not probably be different. On the other hand, age can impact the choice both from an availability point of view (i.e. young or really old may not have a driving license) and a physical effort one.

In [15]:
age = Variable('age')

In [16]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time_walking = Beta('beta_time_walking', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)
ASC_w = Beta('ASC_w', 0, None, None, 0)
ASC_c = Beta('ASC_c', 0, None, None, 0)
ASC_pt = Beta('ASC_pt', 0, None, None, 0)


In [17]:
interacted_ASC_w = ASC_w * age/99
interacted_ASC_c = ASC_c * age/99
interacted_ASC_pt = ASC_pt * age/99


In [18]:
v_w_Model2_ASC = (
    interacted_ASC_w
    + beta_time_walking * dur_walking
)

v_c_Model2_ASC = (
    interacted_ASC_c
    + beta_time_cycling * dur_cycling
)

v_pt_Model2_ASC = (
    interacted_ASC_pt
    + beta_cost * cost_transit
    + beta_time_pt * dur_pt
)

v_driving_Model2_ASC = (
    beta_cost * cost_driving
    + beta_time_driving * dur_driving
)

In [19]:
V_Model2_ASC = {1: v_w_Model2_ASC , 2: v_c_Model2_ASC, 3: v_pt_Model2_ASC, 4: v_driving_Model2_ASC}
logprob_Model2_ASC = models.loglogit(V_Model2_ASC, None, travel_mode)
biogeme_Model2_ASC = bio.BIOGEME(database, logprob_Model2_ASC)
biogeme_Model2_ASC.modelName = 'logit_Model2_ASC'
res_Model2_ASC = biogeme_Model2_ASC.estimate()

In [20]:
print(res_Model2_ASC.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4615.576
Likelihood ratio test for the init. model:	4631.793
Rho-square for the init. model:	0.334
Rho-square-bar for the init. model:	0.333
Akaike Information Criterion:	9247.151
Bayesian Information Criterion:	9299.289
Final gradient norm:	4.2811E-03
Nbr of threads:	8



In [21]:
res_Model2_ASC.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_c,-4.876934,0.374371,-13.027024,0.000000e+00
ASC_pt,-0.998334,0.133146,-7.498069,6.483702e-14
ASC_w,1.531976,0.158595,9.659663,0.000000e+00
beta_cost,-0.150734,0.014566,-10.348652,0.000000e+00
beta_time_cycling,-8.105935,0.665889,-12.173110,0.000000e+00
beta_time_driving,-5.751921,0.391627,-14.687258,0.000000e+00
beta_time_pt,-3.123986,0.244501,-12.776987,0.000000e+00
beta_time_walking,-5.265470,0.197674,-26.637139,0.000000e+00


The estimated signs leaves us satisfied. Moreover, from this estimation we can notice that walking is still the highest among the ASCs, but it is also a lot penalized by its beta_time, which we could expect. Anyway the fact that beta_time_cycling is so low seems pretty strange to us, since we thinik that time has a bigger impact in other contexts such as walking. We can also see that beta_cost has a less signifcant impact on the choices.

- Interaction of age with cost:

When we have to analyze the interaction of the age with one of the variables, we decided to link it with cost, since we think that indviduals may have a differtent perception of money depending on their age. For example, when you're a student you give a different value to money than the one you give them when you've been working for several years.

In [22]:
constant_walking = Beta('constant_walking', 0, None, None, 0)
constant_cycling = Beta('constant_cycling', 0, None, None, 0)
constant_pt = Beta('constant_pt', 0, None, None, 0)
beta_time_walking = Beta('beta_time_walking', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)
beta_cost_int = Beta('beta_cost_int', 0, None, None, 0)

In [23]:
beta_cost_interacted = beta_cost_int * age/99

In [24]:
v_w_Model2_cost = (
    constant_walking
    + beta_time_walking * dur_walking
)

v_c_Model2_cost = (
    constant_cycling
    + beta_time_cycling * dur_cycling
)

v_pt_Model2_cost= (
    constant_pt
    + beta_cost_interacted * cost_transit    
    + beta_time_pt * dur_pt
)

v_driving_Model2_cost = (
    beta_cost_interacted * cost_driving
    + beta_time_driving * dur_driving
)

In [25]:
V_Model2_cost = {1: v_w_Model2_cost, 2: v_c_Model2_cost, 3: v_pt_Model2_cost, 4: v_driving_Model2_cost}
logprob_Model2_cost = models.loglogit(V_Model2_cost, None, travel_mode)
biogeme_Model2_cost = bio.BIOGEME(database, logprob_Model2_cost)
biogeme_Model2_cost.modelName = 'logit_Model2_cost'
res_Model2_cost = biogeme_Model2_cost.estimate()

In [26]:
print(res_Model2_cost.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4387.311
Likelihood ratio test for the init. model:	5088.321
Rho-square for the init. model:	0.367
Rho-square-bar for the init. model:	0.366
Akaike Information Criterion:	8790.622
Bayesian Information Criterion:	8842.76
Final gradient norm:	1.8462E-03
Nbr of threads:	8



In [27]:
res_Model2_cost.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost_int,-0.287265,0.039548,-7.263774,3.763656e-13
beta_time_cycling,-5.430248,0.494069,-10.990875,0.000000e+00
beta_time_driving,-5.958971,0.382057,-15.597075,0.000000e+00
beta_time_pt,-3.218752,0.242768,-13.258575,0.000000e+00
beta_time_walking,-8.180582,0.386817,-21.148430,0.000000e+00
constant_cycling,-2.566973,0.150922,-17.008640,0.000000e+00
constant_pt,-0.407455,0.069331,-5.876925,4.179564e-09
constant_walking,1.939261,0.129410,14.985393,0.000000e+00


The signs we get are still ok for us, then we can also see that this time beta_cost is lower than before, and this is due to the fact that we are considering the interaction with age. We can also see that the ASCs are still pretty much the same. 

Now it's the tome of testing the models. Since the previous preferred model is Model1, and none of our last two models can be seen as a restriction of Model 1, we cannot apply loglikelihood ratio test directly. We instead want to perfrom a Cox test, where we define a third model, as the sum of model 1 and the interaction term. We then perform the likelihood ratio test between the model 1 and the cox model and then between the interacted model and the cox model. The null hypotesis for each of this test is that the restricted model is the better one. We set the significance level of the test to 0,05. If the cox_model results to be the best in both case we have to look for better models, if the models are not rejected in both case we need to change our test and in the other cases we'll be able to state the preferred model. Of course this has to be done for both the interactions we defined. 

In [28]:
v_w_cox1 = v_walk_Model1 + interacted_ASC_w
v_c_cox1 = v_cycling_Model1 + interacted_ASC_c
v_pt_cox1 = v_PT_Model1 + interacted_ASC_pt
v_driving_cox1 = v_driving_Model1

In [29]:
V_cox1 = {1: v_w_cox1, 2: v_c_cox1, 3: v_pt_cox1, 4: v_driving_cox1}
logprob_cox1 = models.loglogit(V_cox1, None, travel_mode)
biogeme_cox1 = bio.BIOGEME(database, logprob_cox1)
biogeme_cox1.modelName = 'logit_cox1'
res_cox1 = biogeme_cox1.estimate()

In [30]:
print(res_cox1.printGeneralStatistics())

Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4348.347
Likelihood ratio test for the init. model:	5166.249
Rho-square for the init. model:	0.373
Rho-square-bar for the init. model:	0.371
Akaike Information Criterion:	8718.694
Bayesian Information Criterion:	8790.383
Final gradient norm:	2.3308E-03
Nbr of threads:	8



In [31]:
res_cox1.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_c,-1.056331,0.365441,-2.890563,3.845519e-03
ASC_pt,-0.959524,0.187752,-5.110597,3.211423e-07
ASC_w,-1.397502,0.236492,-5.909296,3.435729e-09
beta_cost,-0.137210,0.013975,-9.818290,0.000000e+00
beta_time_cycling,-5.425166,0.493299,-10.997715,0.000000e+00
beta_time_driving,-5.742581,0.383501,-14.974113,0.000000e+00
beta_time_pt,-3.113525,0.243493,-12.786929,0.000000e+00
beta_time_walking,-8.186674,0.389671,-21.009213,0.000000e+00
constant_cycling,-2.131890,0.210885,-10.109264,0.000000e+00
constant_pt,-0.012516,0.102428,-0.122195,9.027448e-01


In [38]:
print(res_cox1.likelihood_ratio_test(res_Model1, 0.05))
print(res_cox1.likelihood_ratio_test(res_Model2_ASC, 0.05))

LRTuple(message='H0 can be rejected at level 5.0%', statistic=49.127263927997774, threshold=7.814727903251179)
LRTuple(message='H0 can be rejected at level 5.0%', statistic=534.4566876591743, threshold=7.814727903251179)


In this case both model are rejected and so, as we said, we need to keep looking for other models

In [33]:
v_w_cox2 = v_walk_Model1  
v_c_cox2 = v_cycling_Model1
v_pt_cox2 = v_PT_Model1 + beta_cost_interacted * cost_transit
v_driving_cox2 = v_driving_Model1 + beta_cost_interacted * cost_driving

In [34]:
V_cox2 = {1: v_w_cox2, 2: v_c_cox2, 3: v_pt_cox2, 4: v_driving_cox2}
logprob_cox2 = models.loglogit(V_cox2, None, travel_mode)
biogeme_cox2 = bio.BIOGEME(database, logprob_cox2)
biogeme_cox2.modelName = 'logit_cox2'
res_cox2 = biogeme_cox2.estimate()

In [35]:
print(res_cox2.printGeneralStatistics())

Number of estimated parameters:	9
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4371.267
Likelihood ratio test for the init. model:	5120.411
Rho-square for the init. model:	0.369
Rho-square-bar for the init. model:	0.368
Akaike Information Criterion:	8760.533
Bayesian Information Criterion:	8819.188
Final gradient norm:	8.6372E-04
Nbr of threads:	8



In [36]:
res_cox2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.194854,0.033653,-5.790010,7.038225e-09
beta_cost_int,0.147775,0.079102,1.868163,6.173939e-02
beta_time_cycling,-5.450925,0.493378,-11.048165,0.000000e+00
beta_time_driving,-5.797144,0.384984,-15.058141,0.000000e+00
beta_time_pt,-3.124660,0.244302,-12.790159,0.000000e+00
beta_time_walking,-8.165163,0.386227,-21.140848,0.000000e+00
constant_cycling,-2.559675,0.150780,-16.976168,0.000000e+00
constant_pt,-0.406721,0.069511,-5.851144,4.882045e-09
constant_walking,1.933683,0.129094,14.978860,0.000000e+00


In [37]:
print(res_cox2.likelihood_ratio_test(res_Model1, 0.05))
print(res_cox2.likelihood_ratio_test(res_Model2_cost, 0.05))

LRTuple(message='H0 cannot be rejected at level 5.0%', statistic=3.288482000320073, threshold=3.841458820694124)
LRTuple(message='H0 can be rejected at level 5.0%', statistic=32.08931386668337, threshold=3.841458820694124)


Preferred model is Model 1

# Model 3: #

We would like to apply the boxcox transofrmation to the time variables. Our choice is based on the fact that we think it's more likely to change the perception of time in a non linear way, when the duration of a travel increases, especially in context such as walking or cycling. Anyway, since we want to be sure that our assumption does not bias the model too much, we are going to apply the boxcox transformation which depending on lambda parameter will give us a wider range of correlation between the time and its perception. 

In [42]:
lambda_boxcox_w = Beta('lambda_boxcox_w', 1, -10, 10, 0)
lambda_boxcox_c = Beta('lambda_boxcox_c', 1, -10, 10, 0)
lambda_boxcox_pt = Beta('lambda_boxcox_pt', 1, -10, 10, 0)
lambda_boxcox_driving = Beta('lambda_boxcox_driving', 1, -10, 10, 0)
boxcox_time_w = models.boxcox(dur_walking, lambda_boxcox_w)
boxcox_time_c = models.boxcox(dur_cycling, lambda_boxcox_c)
boxcox_time_pt = models.boxcox(dur_pt, lambda_boxcox_pt)
boxcox_time_driving = models.boxcox(dur_driving, lambda_boxcox_driving)

In [43]:
v_w_Model3 = (
    constant_walking
    + beta_time_walking * boxcox_time_w
)

v_c_Model3 = (
    constant_cycling
    + beta_time_cycling * boxcox_time_c
)

v_pt_Model3= (
    constant_pt
    + beta_cost * cost_transit    
    + beta_time_pt * boxcox_time_pt
)

v_driving_Model3 = (
    beta_cost * cost_driving
    + beta_time_driving * boxcox_time_driving
)

In [44]:
v_Model3 = {1: v_w_Model3, 2: v_c_Model3, 3: v_pt_Model3, 4: v_driving_Model3}
logprob_Model3 = models.loglogit(v_Model3, None, travel_mode)
biogeme_Model3 = bio.BIOGEME(database, logprob_Model3)
biogeme_Model3.modelName = 'logit_Model3'
res_Model3 = biogeme_Model3.estimate()


In [45]:
res_Model3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.121931,0.013848,-8.805163,0.000000e+00
beta_time_cycling,-3.219416,0.508661,-6.329198,2.464391e-10
beta_time_driving,-3.376682,0.296580,-11.385409,0.000000e+00
beta_time_pt,-3.551234,0.337464,-10.523293,0.000000e+00
beta_time_walking,-5.256316,0.379644,-13.845368,0.000000e+00
constant_cycling,-2.329929,0.260703,-8.937104,0.000000e+00
constant_pt,1.539500,0.140732,10.939190,0.000000e+00
constant_walking,0.099235,0.282442,0.351345,7.253293e-01
lambda_boxcox_c,0.279432,0.129578,2.156483,3.104598e-02
lambda_boxcox_driving,0.320108,0.057551,5.562126,2.665073e-08


The estiamted lambdas are quite different, and this leaves us happy, especially the lambda_boxcox_pt is close to 1 that means it's close to be linear (whihc we expected since on public transport you may not feel too much difference between the first minutes and the last ones, since as we've already said you're able to rest or to do other stuffs). Another interesting fact we can notice is that all lambdas are smaller than 1 which means that the more we increase the time the less impact we get on the individual. The signs of the betas are still coherent and the order of ASCs reamins as usual. Looking at t-test values, we see that all of them are gratetr in absoulte value than 1.96, except for ASC_walking, which results to be quite low meaning that this is not really significant.

In [48]:
res_Model3.likelihood_ratio_test(res_Model1, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=190.7572699829543, threshold=9.487729036781154)

We applied the likelihood ratio test since boxcox transformation can be resricted to model 1 (linear) by putting all the lambdas to 1. The result seems good and so now our preferred model in Model3, this confirms our intial assumptions about different perception of time.

# Model 4: #

We choose to test a nested structure. Divding our options in MOTORIZED and UNMOTORIZED (fare grafi su latex). The division seems reasonable to us since when you have to decide a travel mode, at least in our case, the flow of thoughts usually starts from deciding if we want to go with a motorized or unmotorized alternative, depending on how fast do we have to be, on how do we phisically fell and on other factors such as weather conditions.
We also thought about a division between public and private transport choices but we prefered the above mentioned one since public transport modes are all put together under a single alternative and hence the division would not be a lot siginficant

In [ ]:
motorized = Beta('motorized', 0, None, None, 0)